In [62]:
from datetime import datetime,date
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Merge and clean Activity and Sleep datasets from 2020
* Will create two csv files: one with outliers and one without outliers
* Units of measurment that represnt time are in hours. For example, 10.33 would be 10 hours and 33 minutes.

In [63]:
#Load and append both activity datasets
a_first_half = pd.read_csv('C:\\Users\jwkon\Data Learning\FitData\Fit_Project(2020)\Raw_Data\Clean_Activity_Data(Jan-May).csv')
a_second_half = pd.read_csv('C:\\Users\jwkon\Data Learning\FitData\Fit_Project(2020)\Raw_Data\Clean_Activity_Data(Jun-Dec).csv')
activity = a_first_half.append(a_second_half)
activity.reset_index(drop=True, inplace=True)
#Load and append both sleep datasets
s_first_half = pd.read_csv('C:\\Users\jwkon\Data Learning\FitData\Fit_Project(2020)\Raw_Data\Clean_Sleep_Data(Jan-May).csv')
s_second_half = pd.read_csv('C:\\Users\jwkon\Data Learning\FitData\Fit_Project(2020)\Raw_Data\Clean_Sleep_Data(Jun-Dec).csv')
sleep = s_first_half.append(s_second_half)
sleep.reset_index(drop=True, inplace=True)

In [64]:
#View column quality and data types
activity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Date                              360 non-null    object 
 1   Calories Burned                   360 non-null    int64  
 2   Steps                             360 non-null    int64  
 3   Distance                          360 non-null    float64
 4   Floors                            360 non-null    int64  
 5   Minutes Sedentary                 360 non-null    int64  
 6   Minutes Lightly Active(Fat Burn)  360 non-null    int64  
 7   Minutes Fairly Active(Cardio)     360 non-null    int64  
 8   Minutes Very Active(Peak)         360 non-null    int64  
 9   Activity Calories                 360 non-null    int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 28.2+ KB


In [65]:
#View column quality and data types
sleep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Start Time            363 non-null    object 
 1   End Time              363 non-null    object 
 2   Minutes Asleep        363 non-null    int64  
 3   Minutes Awake         363 non-null    int64  
 4   Number of Awakenings  363 non-null    int64  
 5   Time in Bed           363 non-null    int64  
 6   Minutes REM Sleep     363 non-null    float64
 7   Minutes Light Sleep   363 non-null    float64
 8   Minutes Deep Sleep    363 non-null    float64
dtypes: float64(3), int64(4), object(2)
memory usage: 25.6+ KB


In [66]:
#Preview activity data
activity.head(1)

,Date,Calories Burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active(Fat Burn),Minutes Fairly Active(Cardio),Minutes Very Active(Peak),Activity Calories
0,2020-01-01,2736,10201,4.46,6,633,341,10,24,1482


In [67]:
#Preview sleep data
sleep.head(1)

,Start Time,End Time,Minutes Asleep,Minutes Awake,Number of Awakenings,Time in Bed,Minutes REM Sleep,Minutes Light Sleep,Minutes Deep Sleep
0,2020-01-01 01:16:00,2020-01-01 08:28:00,334,98,20,432,47.0,242.0,45.0


In [68]:
#Convert sleep Time columns to datetime format and group on End Time
sleep['Start Time'] = pd.to_datetime(sleep['Start Time']).dt.date
sleep['End Time'] = pd.to_datetime(sleep['End Time']).dt.date

In [69]:
#Convert activity date column to datetime format and group on Date
activity['Date'] = pd.to_datetime(activity['Date']).dt.date
activity = activity.groupby(['Date']).agg(sum).reset_index()

In [70]:
#Merge dataframes on Date & End Time
data = pd.merge(activity, sleep, left_on=['Date'], right_on=['End Time'])

In [71]:
#Identify and remove outliers
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

data = data[~((data < (Q1 - 1.5 * IQR)) |(data > (Q3 + 1.5 * IQR))).any(axis=1)]

In [72]:
#Set Date column to datetime format/data type
data['Date'] = pd.to_datetime(data['Date'])

In [73]:
#Function that converts the column values from raw minutes to a more familiar format of hours with minutes as a decimal
def toHour(minutes = [], *args):
    for i in minutes:
        data[i] = pd.to_datetime(data[i], unit='m').dt.strftime('%H:%M')
        data[i] = data[i].replace(':','.', regex=True)
        data[i] = data[i].astype(str).astype(float)

In [74]:
#Invoke toHour function
toHour(['Minutes Sedentary', 'Minutes Lightly Active(Fat Burn)', 'Minutes Asleep', 'Minutes Awake', 'Time in Bed', 'Minutes REM Sleep','Minutes Light Sleep', 'Minutes Deep Sleep'])

In [75]:
#Rename column headers
data.rename(columns={'Distance': 'Distance(Miles)', 'Minutes Sedentary':'Time Sedentary', 'Minutes Lightly Active(Fat Burn)': 'Lightly Active(Fat Burn)', 'Minutes Fairly Active(Cardio)': 'Fairly Active(Cardio)', 'Minutes Very Active(Peak)': 'Very Active(Peak)', 'Minutes Asleep': 'Time Asleep', 'Minutes Awake': 'Time Awake', 'Minutes REM Sleep': 'REM Sleep', 'Minutes Light Sleep': 'Light Sleep', 'Minutes Deep Sleep': 'Deep Sleep'}, inplace=True)

In [76]:
#Drop rows that have duplicates in the Date column since that represents each day with full data
data = data.drop_duplicates(subset=['Date'], keep='first')

In [77]:
#Verify column quality and data types
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 276 entries, 0 to 360
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      276 non-null    datetime64[ns]
 1   Calories Burned           276 non-null    int64         
 2   Steps                     276 non-null    int64         
 3   Distance(Miles)           276 non-null    float64       
 4   Floors                    276 non-null    int64         
 5   Time Sedentary            276 non-null    float64       
 6   Lightly Active(Fat Burn)  276 non-null    float64       
 7   Fairly Active(Cardio)     276 non-null    int64         
 8   Very Active(Peak)         276 non-null    int64         
 9   Activity Calories         276 non-null    int64         
 10  Start Time                276 non-null    object        
 11  End Time                  276 non-null    object        
 12  Time Asleep           

In [78]:
#View first 5 rows
data.head()

,Date,Calories Burned,Steps,Distance(Miles),Floors,Time Sedentary,Lightly Active(Fat Burn),Fairly Active(Cardio),Very Active(Peak),Activity Calories,Start Time,End Time,Time Asleep,Time Awake,Number of Awakenings,Time in Bed,REM Sleep,Light Sleep,Deep Sleep
0,2020-01-01,2736,10201,4.46,6,10.33,5.41,10,24,1482,2020-01-01,2020-01-01,5.34,1.38,20,7.12,0.47,4.02,0.45
1,2020-01-02,2637,9539,4.25,4,10.08,4.52,31,2,1302,2020-01-01,2020-01-02,6.54,1.33,33,8.27,0.50,5.46,0.18
2,2020-01-03,2656,11394,4.75,5,12.30,4.02,32,27,1328,2020-01-02,2020-01-03,5.31,0.58,27,6.29,0.31,4.38,0.22
3,2020-01-04,2934,17150,7.20,6,9.01,4.54,16,36,1657,2020-01-03,2020-01-04,7.44,1.29,36,9.13,1.24,5.41,0.39
5,2020-01-06,2174,7296,3.23,5,13.33,2.43,15,9,782,2020-01-05,2020-01-06,6.06,1.14,23,7.20,0.59,4.22,0.45


In [79]:
#View last 5 rows
data.tail()

,Date,Calories Burned,Steps,Distance(Miles),Floors,Time Sedentary,Lightly Active(Fat Burn),Fairly Active(Cardio),Very Active(Peak),Activity Calories,Start Time,End Time,Time Asleep,Time Awake,Number of Awakenings,Time in Bed,REM Sleep,Light Sleep,Deep Sleep
355,2020-12-19,2352,6858,3.03,6,12.51,4.50,0,0,1021,2020-12-18,2020-12-19,5.19,1.00,22,6.19,1.06,4.09,0.04
356,2020-12-20,2489,4726,2.10,0,12.29,5.18,10,0,1159,2020-12-19,2020-12-20,5.11,0.52,18,6.03,1.01,3.30,0.40
357,2020-12-21,2416,7184,3.16,2,14.16,4.26,19,4,1100,2020-12-21,2020-12-21,4.09,0.46,14,4.55,0.54,2.45,0.30
358,2020-12-22,2217,5625,2.50,7,12.23,4.13,0,0,859,2020-12-21,2020-12-22,6.26,0.58,29,7.24,1.24,3.51,1.11
360,2020-12-24,2609,7911,3.51,2,11.08,5.43,0,0,1273,2020-12-23,2020-12-24,6.05,0.52,32,6.57,1.32,3.41,0.52


In [80]:
#View summary statistics
data.describe()

,Calories Burned,Steps,Distance(Miles),Floors,Time Sedentary,Lightly Active(Fat Burn),Fairly Active(Cardio),Very Active(Peak),Activity Calories,Time Asleep,Time Awake,Number of Awakenings,Time in Bed,REM Sleep,Light Sleep,Deep Sleep
count,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000,276.000000
mean,2426.000000,7796.141304,3.723841,8.021739,12.362065,3.761957,12.942029,15.315217,1068.028986,5.559203,0.806630,23.909420,6.574529,0.815870,3.762029,0.587572
std,325.269634,3575.180144,1.970808,7.025633,1.735667,1.017042,14.353343,20.125748,384.344929,1.208215,0.346821,6.491599,1.406252,0.387086,0.894674,0.343786
min,1792.000000,1746.000000,0.780000,0.000000,7.460000,1.260000,0.000000,0.000000,302.000000,2.200000,0.150000,3.000000,2.460000,0.080000,1.430000,0.040000
25%,2179.500000,4924.250000,2.190000,3.000000,11.257500,3.170000,0.000000,0.000000,782.750000,5.057500,0.490000,19.750000,5.545000,0.467500,3.270000,0.360000
50%,2389.500000,7073.500000,3.230000,6.000000,12.320000,4.010000,9.000000,4.000000,1045.000000,5.535000,0.795000,24.000000,6.570000,1.000000,4.025000,0.490000
75%,2657.500000,10562.250000,4.970000,10.250000,13.382500,4.370000,20.000000,28.000000,1331.250000,6.322500,1.110000,29.000000,7.380000,1.170000,4.330000,1.000000
max,3460.000000,20162.000000,10.420000,30.000000,16.350000,6.340000,62.000000,82.000000,2259.000000,8.370000,1.470000,39.000000,9.550000,1.560000,6.070000,1.330000


In [81]:
#Save to csv for analysis
data.to_csv('FitData_2020(OutliersRemoved).csv', index=False)